Импорт модулей 

In [1]:
import psycopg2
import pandas as pd
import plotly.graph_objects as go
from plotly.data import tips

Подключение к БД для загрузки данных

In [2]:
conn = psycopg2.connect(
    host = 'XXX.XXX.XXX.XXX',
    port = XXXX,
    database = 'postgres',
    user = 'XXXX',
    password = 'XXXX'
)

Отключение автокоммитов, чтобы не нагружать сервер

In [3]:
conn.autocommit = False

Создание курсора для данных по месяцам из SQL запроса

In [4]:
cursor_m = conn.cursor()

In [5]:
cursor_m.execute('''
SELECT TO_CHAR((visit_dttm), 'YYYY-MM') AS months
       , COUNT(DISTINCT c.client_rk) AS visiters_site
       , COUNT(DISTINCT a1.account_rk) AS registration_site
       , COUNT(DISTINCT a2.account_rk) AS reserve_time
       , COUNT(DISTINCT a2.account_rk) FILTER(WHERE g.game_flg = 1) AS fact_game 
  FROM msu_analytics.client AS c
       LEFT JOIN msu_analytics.account AS a1
            ON c.client_rk = a1.client_rk
       LEFT JOIN msu_analytics.application AS a2
            ON a1.account_rk = a2.account_rk
       LEFT JOIN msu_analytics.game AS g
            ON a2.game_rk  = g.game_rk
 GROUP BY months
 ORDER BY months
''')

Возвращение результата и формирование датафрейма (имена полей из описания)

In [6]:
result_m = cursor_m.fetchall()
names_m = [ x[0] for x in cursor_m.description ]
df_m = pd.DataFrame( result_m, columns = names_m )

Создание курсора для данных по неделям из SQL запроса

In [7]:
cursor_w = conn.cursor()

In [8]:
cursor_w.execute('''
SELECT CAST(DATE_TRUNC('WEEK', visit_dttm) AS DATE) AS weeks
       , COUNT(DISTINCT c.client_rk) AS visiters_site
       , COUNT(DISTINCT a1.account_rk) AS registration_site
       , COUNT(DISTINCT a2.account_rk) AS reserve_time
       , COUNT(DISTINCT a2.account_rk) FILTER(WHERE g.game_flg = 1) AS fact_game 
  FROM msu_analytics.client AS c
       LEFT JOIN msu_analytics.account AS a1
            ON c.client_rk = a1.client_rk
       LEFT JOIN msu_analytics.application AS a2
            ON a1.account_rk = a2.account_rk
       LEFT JOIN msu_analytics.game AS g
            ON a2.game_rk  = g.game_rk
 GROUP BY weeks
 ORDER BY weeks
''')

Возвращение результата и формирование датафрейма (имена полей из описания)

In [9]:
result_w = cursor_w.fetchall()
names_w = [ x[0] for x in cursor_w.description ]
df_w = pd.DataFrame( result_w, columns = names_w )

Добавление полей с расчетом конверсии и визуальный контроль полученного датафрейма

In [10]:
df_m.insert(3, "persent_reg", round(df_m['registration_site'] / df_m['visiters_site'] * 100, 2))
df_m.insert(5, "persent_res", round(df_m['reserve_time'] / df_m['registration_site'] * 100, 2))
df_m.insert(7, "persent_game", round(df_m['fact_game'] / df_m['reserve_time'] * 100, 2))

In [11]:
df_w.insert(3, "persent_reg", round(df_w['registration_site'] / df_w['visiters_site'] * 100, 2))
df_w.insert(5, "persent_res", round(df_w['reserve_time'] / df_w['registration_site'] * 100, 2))
df_w.insert(7, "persent_game", round(df_w['fact_game'] / df_w['reserve_time'] * 100, 2))

In [12]:
df_m.head()

,months,visiters_site,registration_site,persent_reg,reserve_time,persent_res,fact_game,persent_game
0,2022-09,778,190,24.42,51,26.84,39,76.47
1,2022-10,843,217,25.74,65,29.95,56,86.15
2,2022-11,786,200,25.45,56,28.00,50,89.29
3,2022-12,805,196,24.35,57,29.08,33,57.89
4,2023-01,852,213,25.00,64,30.05,20,31.25


In [13]:
df_w

,weeks,visiters_site,registration_site,persent_reg,reserve_time,persent_res,fact_game,persent_game
0,2022-08-29,111,32,28.83,7,21.88,5,71.43
1,2022-09-05,184,31,16.85,6,19.35,5,83.33
2,2022-09-12,167,43,25.75,16,37.21,8,50.00
3,2022-09-19,181,45,24.86,12,26.67,11,91.67
4,2022-09-26,195,59,30.26,17,28.81,17,100.00
5,2022-10-03,204,58,28.43,18,31.03,15,83.33
6,2022-10-10,190,44,23.16,13,29.55,13,100.00
7,2022-10-17,171,41,23.98,12,29.27,10,83.33
8,2022-10-24,189,48,25.40,15,31.25,11,73.33
9,2022-10-31,197,47,23.86,16,34.04,14,87.50


**Создание визуализации**

Из протестированных графиков, выбран наиболее читаемый.

Формирование визуализации с помесячным отображением данных и с еженедельным.

In [14]:
fig_m = go.Figure(
    data=go.Bar(
        x=df_m['months'],
        y=df_m['visiters_site'],
        name="Посетители сайта",
    )
)

fig_m.add_bar(
        x=df_m['months'],
        y=df_m['registration_site'],
        name="Зарегистрировавшиеся",
)

fig_m.add_bar(
        x=df_m['months'],
        y=df_m['reserve_time'],
        name="Подавшие заявку",
)

fig_m.add_bar(
        x=df_m['months'],
        y=df_m['fact_game'],
        name="Сыгравшие",
)

fig_m.add_trace(
    go.Scatter(
        x=df_m['months'],
        y=df_m['persent_reg'],
        yaxis="y2",
        name="Конверсия посетителей в зарегистрировавшихся",
    )
)

fig_m.add_trace(
    go.Scatter(
        x=df_m['months'],
        y=df_m['persent_res'],
        yaxis="y2",
        name="Конверсия из зарегистрированных в подачу заявки на игру",
    )
)

fig_m.add_trace(
    go.Scatter(
        x=df_m['months'],
        y=df_m['persent_game'],
        yaxis="y2",
        name="Конверсия поданных заявок в сыгранные игры",
    )
)

fig_m.update_layout(
    legend=dict(orientation="h"),
    yaxis=dict(
        title=dict(text="Количество клиентов"),
        side="left",
        range=[0, 1000],
    ),
    yaxis2=dict(
        title=dict(text="Проценты"),
        side="right",
        range=[0, 100],
        overlaying="y",
        tickmode="sync",
    ),
)

fig_m.show()

**Вывод:**

1. Вывод можно делать с сентября по январь включительно, т.к. данных за февраль недостаточно.
2. Посещаемость сайта держится на одном уровне. Требуется увеличивать посещаемость за счет рекламы и продвижения.
3. Конверсия в зарегистрировавшихся держится на одном уровне. Требуется привлечение клиента на сайте, например, бонусами на первую игру за регистрацию при первом посещении сайта.
4. Конверсия в тех, кто подает заявку на игру стабильна. Привлечение за счет рассылки зарегистрировавшимся рекламы игр и предложений скидок на непопулярные часы посещения.
5. Конверсия поданных заявок в сыгранные игры высокая, но в декабре и январе значительно снижается. Данный факт можно списать на загруженность на работе в декабре и подготовкой клиентов к праздникам, следовательно принимают решение отменить посещение игры. В январе может сказываться сокращение трат после праздников. Предлагается клиентам бесплатный перенос игры до 45 дней. Возможно предложение рассрочки оплаты до 60 дней.

In [15]:
fig_w = go.Figure(
    data=go.Bar(
        x=df_w['weeks'],
        y=df_w['visiters_site'],
        name="Посетители сайта",
    )
)

fig_w.add_bar(
        x=df_w['weeks'],
        y=df_w['registration_site'],
        name="Зарегистрировавшиеся",
)

fig_w.add_bar(
        x=df_w['weeks'],
        y=df_w['reserve_time'],
        name="Подавшие заявку",
)

fig_w.add_bar(
        x=df_w['weeks'],
        y=df_w['fact_game'],
        name="Сыгравшие",
)

fig_w.add_trace(
    go.Scatter(
        x=df_w['weeks'],
        y=df_w['persent_reg'],
        yaxis="y2",
        name="Конверсия посетителей в зарегистрировавшихся",
    )
)

fig_w.add_trace(
    go.Scatter(
        x=df_w['weeks'],
        y=df_w['persent_res'],
        yaxis="y2",
        name="Конверсия из зарегистрированных в подачу заявки на игру",
    )
)

fig_w.add_trace(
    go.Scatter(
        x=df_w['weeks'],
        y=df_w['persent_game'],
        yaxis="y2",
        name="Конверсия поданных заявок в сыгранные игры",
    )
)

fig_w.update_layout(
    legend=dict(orientation="h"),
    yaxis=dict(
        title=dict(text="Количество клиентов"),
        side="left",
        range=[0, 220],
    ),
    yaxis2=dict(
        title=dict(text="Проценты"),
        side="right",
        range=[0, 100],
        overlaying="y",
        tickmode="sync",
    ),
)

fig_w.show()

**Вывод:**

1. После анализа визуализации по месяцам, выводы по недельной визуализации подтверждаются.
2. Первая и последняя не рассматриваются, т.к. неполное заполнение днями месяцев.
3. На третьей неделе значительное снижение конверсии подданных заявок в сыгранные игры, при повышенной конверсии подачи заявок среди зарегистрировавшихся. Возможно, связано с началом учебного года, последними теплыми днями сезона.
4. Три недели достигают конверсии в сыгранные игры 100%.
5. Требуется дополнительный анализ падения конверсии из зарегистрированных в подачу заявок в середине ноября.
